<a href="https://colab.research.google.com/github/JatinSharma496/mlproject/blob/main/OneBlance_Data_Science_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**OneBlanc Assigment**


***MPIN Strength Checker Notebook***


This notebook walks through the process of building and testing an MPIN strength checker based on the assignment requirements.






**OneBlanc Assigment**


In [2]:
config = {
    "COMMONLY_USED_MPINS_4": [
        "1234", "0000", "1111", "2222", "3333", "4444", "5555", "6666",
        "7777", "8888", "9999", "1122", "1212", "1313", "6969", "4321",
        "1004", "2000", "2580", "0987", "1010"
    ],
    "COMMONLY_USED_MPINS_6": [
        "123456", "000000", "111111", "654321", "121212", "112233",
        "999999", "159753", "123123", "789456", "101010", "123321",
        "456789", "121314", "111222", "314159", "777777", "010101"
    ]
}

print("Configuration loaded successfully.")


Configuration loaded successfully.


In [6]:

class MPINStrengthChecker:
    """
    A class to check the strength of a 4 or 6-digit MPIN based on common patterns
    and personal demographic data.
    """

    def __init__(self, common_pins_config):
        """Initializes the checker with sets of commonly used MPINs."""
        self.common_pins_4 = set(common_pins_config.get("COMMONLY_USED_MPINS_4", []))
        self.common_pins_6 = set(common_pins_config.get("COMMONLY_USED_MPINS_6", []))
        print("MPINStrengthChecker initialized.")

    def is_commonly_used(self, mpin):
        """Public method to check if the MPIN is commonly used (Part A)."""
        if len(mpin) == 4:
            return mpin in self.common_pins_4
        elif len(mpin) == 6:
            return mpin in self.common_pins_6
        return False

    def _extract_date_combinations(self, date_str):
        """Extracts various 4 and 6-digit combinations from a date string (DD-MM-YYYY)."""
        if not date_str or len(date_str.split('-')) != 3:
            return set()
        try:
            day, month, year = date_str.split('-')
            if len(day) != 2 or len(month) != 2 or len(year) != 4: return set()
            year_short = year[2:]
        except (ValueError, IndexError):
            return set()

        combinations = {
            # 4-Digit
            day + month, month + day, day + year_short, year_short + day,
            month + year_short, year_short + month, day * 2, month * 2, year_short * 2,
            # 6-Digit
            day + month + year_short, day + year_short + month, month + day + year_short,
            month + year_short + day, year_short + day + month, year_short + month + day
        }
        return combinations

    def check_strength_with_reasons(self, mpin, dob_self=None, dob_spouse=None, anniversary=None):
        """Analyzes MPIN strength and provides reasons if weak (Parts B, C, D)."""
        if not isinstance(mpin, str) or not mpin.isdigit() or len(mpin) not in [4, 6]:
            return "INVALID_LENGTH", []

        reasons = []
        if self.is_commonly_used(mpin):
            reasons.append("COMMONLY_USED")

        demographic_map = {
            "DEMOGRAPHIC_DOB_SELF": dob_self,
            "DEMOGRAPHIC_DOB_SPOUSE": dob_spouse,
            "DEMOGRAPHIC_ANNIVERSARY": anniversary
        }
        for reason_code, date_str in demographic_map.items():
            if date_str and mpin in self._extract_date_combinations(date_str):
                reasons.append(reason_code)

        if reasons:
            return "WEAK", sorted(list(set(reasons)))
        return "STRONG", []

# Instantiate the checker
mpin_checker = MPINStrengthChecker(config)


MPINStrengthChecker initialized.


In [7]:
print("======================================================")
print("         MPIN Strength Checker Demonstrations         ")
print("======================================================")

# --- Part A: Suggest if a 4-digit MPIN is commonly used ---
print("\n--- Part A: Common 4-Digit PIN Check ---")
for pin in ["1111", "5432"]:
    is_common = mpin_checker.is_commonly_used(pin)
    print(f"Is '{pin}' common? {'Yes, weak.' if is_common else 'No.'}")

# --- Part B: Basic Strength (WEAK/STRONG) with Demographics ---
print("\n--- Part B: Basic Strength Check (WEAK/STRONG) ---")
strength, _ = mpin_checker.check_strength_with_reasons("0201", dob_self="02-01-1998")
print(f"PIN '0201' vs DOB '02-01-1998'. Strength: {strength}")
strength, _ = mpin_checker.check_strength_with_reasons("9876", dob_self="02-01-1998")
print(f"PIN '9876' vs DOB '02-01-1998'. Strength: {strength}")

# --- Part C & D: Strength with Reasons (4 & 6-Digit) ---
print("\n--- Part C & D: Strength Check with Reasons ---")
# Part C (4-digit)
s, r = mpin_checker.check_strength_with_reasons("1212", dob_self="12-12-2000")
print(f"Part C -> PIN '1212' -> Strength: {s}, Reasons: {r}")
# Part D (6-digit)
s, r = mpin_checker.check_strength_with_reasons("150815", anniversary="15-08-2015")
print(f"Part D -> PIN '150815' -> Strength: {s}, Reasons: {r}")


         MPIN Strength Checker Demonstrations         

--- Part A: Common 4-Digit PIN Check ---
Is '1111' common? Yes, weak.
Is '5432' common? No.

--- Part B: Basic Strength Check (WEAK/STRONG) ---
PIN '0201' vs DOB '02-01-1998'. Strength: WEAK
PIN '9876' vs DOB '02-01-1998'. Strength: STRONG

--- Part C & D: Strength Check with Reasons ---
Part C -> PIN '1212' -> Strength: WEAK, Reasons: ['COMMONLY_USED', 'DEMOGRAPHIC_DOB_SELF']
Part D -> PIN '150815' -> Strength: WEAK, Reasons: ['DEMOGRAPHIC_ANNIVERSARY']


In [9]:
def run_tests(checker):
    print("\n\n======================================================")

    print("======================================================")
    test_cases = [
        {"pin": "1234", "expected_strength": "WEAK", "expected_reasons": ["COMMONLY_USED"]},
        {"pin": "123456", "expected_strength": "WEAK", "expected_reasons": ["COMMONLY_USED"]},
        {"pin": "0201", "dob_self": "02-01-1998", "expected_strength": "WEAK", "expected_reasons": ["DEMOGRAPHIC_DOB_SELF"]},
        {"pin": "020198", "dob_self": "02-01-1998", "expected_strength": "WEAK", "expected_reasons": ["DEMOGRAPHIC_DOB_SELF"]},
        {"pin": "2512", "dob_spouse": "25-12-1990", "expected_strength": "WEAK", "expected_reasons": ["DEMOGRAPHIC_DOB_SPOUSE"]},
        {"pin": "150815", "anniversary": "15-08-2015", "expected_strength": "WEAK", "expected_reasons": ["DEMOGRAPHIC_ANNIVERSARY"]},
        {"pin": "1212", "dob_self": "12-12-2000", "expected_strength": "WEAK", "expected_reasons": ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"]},
        {"pin": "1005", "dob_self": "10-05-1985", "anniversary": "10-05-2010", "expected_strength": "WEAK", "expected_reasons": ["DEMOGRAPHIC_ANNIVERSARY", "DEMOGRAPHIC_DOB_SELF"]},
        {"pin": "123", "expected_strength": "INVALID_LENGTH", "expected_reasons": []},
        {"pin": "9845", "dob_self": "01-01-1990", "expected_strength": "STRONG", "expected_reasons": []},
        {"pin": "736418", "expected_strength": "STRONG", "expected_reasons": []},
    ] # Abridged for brevity

    passed_count = 0
    for i, test in enumerate(test_cases, 1):
        strength, reasons = checker.check_strength_with_reasons(
            test.get("pin"), test.get("dob_self"), test.get("dob_spouse"), test.get("anniversary")
        )
        expected_reasons = sorted(test.get("expected_reasons", []))
        if strength == test.get("expected_strength") and reasons == expected_reasons:
            status = "✅ Passed"
            passed_count += 1
        else:
            status = f"❌ Failed"
            print(f"\nTest #{i}: PIN='{test.get('pin')}' -> {status}")
            print(f"  -> Expected: {test.get('expected_strength')}, {expected_reasons}")
            print(f"  -> Got:      {strength}, {reasons}")

    print("\n--- Test Summary ---")
    print(f"Total Tests: {len(test_cases)}")
    print(f"Passed:      {passed_count}")
    print(f"Failed:      {len(test_cases) - passed_count}")
    print("--------------------")

# Assuming mpin_checker is instantiated from the previous cell
run_tests(mpin_checker)





--- Test Summary ---
Total Tests: 11
Passed:      11
Failed:      0
--------------------
